# Libbitcoin SPV Client: Validating TX Confirmation

In this example, we demonstrate how to subscribe the SPV client can validate the confirmation of a transaction with a merkle proof.

<br>
<img src="images/spv_merkle_proof.jpg" alt="drawing" style="" width="800px"/>



### Libbitcoin-System (C++) Setup

In [2]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System
// Libbitcoin-Protocol
#include <bitcoin/bitcoin.hpp> 
#include <bitcoin/protocol.hpp> 

<hr style="border: 0.5px dashed #000;">


## 1) Transaction hash to be validated.

* `TX hash` and `chain height` for which merkle proof is to be computed.

In [3]:
auto my_tx_hash = bc::base16_literal("8722a9a073fc4d7d27cf07540ddee2d4449f36d1470eddedb8bd8a70ded88fe3");
std::reverse(my_tx_hash.begin(), my_tx_hash.end());

uint32_t my_tx_chain_height = 1412061;


## 2) Request merkle root at relevant chain height.


### 2.1) Setup ZMQ Socket.


In [4]:
bc::protocol::zmq::context my_context(true); // true = started
bc::protocol::zmq::socket my_requester(my_context,
    bc::protocol::zmq::socket::role::requester);
bc::config::endpoint public_endpoint("tcp://testnet1.libbitcoin.net:19091");

bc::code ec_socket;
ec_socket = my_requester.connect(public_endpoint);


### 2.2) Submit request / Receive requested header.

* Header request message:
    * `[-- "blockchain.fetch_block_header" --]`
    * `[--------- 4-byte message id ---------]`
    * `[--------- 4-byte chain height -------]`
<br><br>

* Reply message:
    * `[------- "blockchain.fetch_block_header" ------]`
    * `[----------- sent 4-byte message id -----------]`
    * `[-- 4-byte error code | 80-byte block header --]`


In [5]:
std::string command = "blockchain.fetch_block_header";
uint32_t message_id(0);
uint32_t requested_height(my_tx_chain_height);

// Client request message.
bc::protocol::zmq::message header_request;
header_request.enqueue(bc::to_chunk(command));
header_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id)));
header_request.enqueue(bc::to_chunk(bc::to_little_endian(requested_height)));

// Server response message.
bc::protocol::zmq::message server_response;
std::string my_message_command;
uint32_t my_message_id;  
bc::data_chunk reply_payload; 
bc::code ec_reply;

// Submit Query.
//------------------------------------------------------------------------------
ec_socket = header_request.send(my_requester);


// Parse Server Reply.
//------------------------------------------------------------------------------
server_response.receive(my_requester); // Blocking until message available.

my_message_command = server_response.dequeue_text();
server_response.dequeue(my_message_id);
server_response.dequeue(reply_payload);

bc::data_source reply_byte_stream(reply_payload);
bc::istream_reader reply_byte_stream_reader(reply_byte_stream);

// Error Code
ec_reply = reply_byte_stream_reader.read_error_code();

// Header
bc::chain::header requested_header;
bc::data_chunk header_data;
requested_header.from_data(reply_byte_stream_reader); // remaining bytes

// Print out merkle root.
auto requested_merkle_root = requested_header.merkle();
std::cout << "Requested Merkle Root: "
          << bc::encode_base16(bc::to_chunk(requested_merkle_root))
          << std::endl;


Requested Merkle Root: 34ee2fe5c46532ef2d9ccde9ce2fe2cc296454f3861f54c87ee3939808877483


## 3) Validate merkle root from TX hash list at relevant chain height.



### 3.1) Request TX hashes at relevant chain height.

* TX hash list request message:
    * `[-- "blockchain.fetch_block_transaction_hashes" --]`
    * `[--------------- 4-byte message id ---------------]`
    * `[--------------- 4-byte chain height -------------]`
<br><br>

* Reply message:
    * `[--------- "blockchain.fetch_block_transaction_hashes" -------]`
    * `[------------------ sent 4-byte message id -------------------]`
    * `[-- 4-byte error code | TX hash 0 | TX hash 1 | TX hash ... --]`
<br><br>
* Use `bc::hash_list` for returned TX hashes.

* **Validate that relevant TX hash is in the list of returned TX hashes.**   

In [6]:
command = "blockchain.fetch_block_transaction_hashes";
message_id += 1;

// Client request message.
bc::protocol::zmq::message hash_list_request;
hash_list_request.enqueue(bc::to_chunk(command));
hash_list_request.enqueue(bc::to_chunk(bc::to_little_endian(message_id)));
hash_list_request.enqueue(bc::to_chunk(bc::to_little_endian(my_tx_chain_height)));

// Submit Query.
//------------------------------------------------------------------------------
ec_socket = hash_list_request.send(my_requester);

// Parse Server Reply.
//------------------------------------------------------------------------------
server_response.receive(my_requester);

bc::data_chunk reply_payload_;
my_message_command = server_response.dequeue_text();
server_response.dequeue(my_message_id);
server_response.dequeue(reply_payload_);

bc::data_source reply_byte_stream_(reply_payload_);
bc::istream_reader reply_byte_stream_reader_(reply_byte_stream_);

ec_reply = reply_byte_stream_reader_.read_error_code();

// read out requested tx hashes.
// create hash list:
bc::hash_list returned_hash_list;
uint32_t tx_block_index(0);
bool tx_hash_found(false);

while (!reply_byte_stream_reader_.is_exhausted())
{
    bc::hash_digest transaction_hash = reply_byte_stream_reader_.read_hash();
    returned_hash_list.push_back(transaction_hash);

    // If TX hash is inside, store state.
    if (my_tx_hash == transaction_hash)
    {
        tx_hash_found = true;
        std::cout << "TX hash found at block index: " << tx_block_index
                  << std::endl;
    }
    tx_block_index += 1;
}

if (!tx_hash_found)
{
    std::cout << "TX hash not found at height: " << requested_height
              << std::endl;
    return 0;
}


TX hash found at block index: 763


### 3.2) Compute and validate merkle root.

In [7]:
bc::hash_list update;
auto merkle = returned_hash_list;

// Initial capacity is half of the original list (clear doesn't reset).
update.reserve((merkle.size() + 1) / 2);

while (merkle.size() > 1)
{
    // If number of hashes is odd, duplicate last hash in the list.
    if (merkle.size() % 2 != 0)
        merkle.push_back(merkle.back());

    for (auto it = merkle.begin(); it != merkle.end(); it += 2)
        update.push_back(bc::bitcoin_hash(bc::build_chunk({ it[0], it[1] })));

    std::swap(merkle, update);
    update.clear();
}

std::cout << "Computed Merkle Root: "
          << bc::encode_base16(bc::to_chunk(merkle.front())) << std::endl;

if(requested_merkle_root == merkle.front())
    std::cout << "Merkle proof validated." << std::endl;


Computed Merkle Root: 34ee2fe5c46532ef2d9ccde9ce2fe2cc296454f3861f54c87ee3939808877483
Merkle proof validated.
